In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 1: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 512
test_batch = 512
lr = 0.1
schedule = [75, 175, 250]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style2/128/b0/aug' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.2
cm_beta = 1.0

# augmentation
blur_prob = 0.2
blog_sig = 0.5
jpg_prob = 0.2

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [11]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


# Loss

In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Train AUROC.', 'Valid AUROC.'])

# Train

In [14]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        arc.update(auroc, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        if batch_idx % 100 == 0:
            print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [15]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
         batch=batch_idx+1, size=len(val_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [16]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [17]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, train_auroc, test_auroc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 300] LR: 0.100000
1/152 | Loss:0.7040 | top1:47.2656 | AUROC:0.4984
101/152 | Loss:2.7218 | top1:50.9031 | AUROC:0.5029
152/152 | Loss:2.0512 | top1:51.4099 | AUROC:0.5030
16/16 | Loss:0.6932 | top1:49.9359 | AUROC:0.5000

Epoch: [2 | 300] LR: 0.170000
1/152 | Loss:0.6966 | top1:50.5859 | AUROC:0.4998
101/152 | Loss:0.6968 | top1:52.9394 | AUROC:0.4973
152/152 | Loss:0.6955 | top1:53.0086 | AUROC:0.4978
16/16 | Loss:0.7010 | top1:50.0000 | AUROC:0.5000

Epoch: [3 | 300] LR: 0.240000
1/152 | Loss:0.6970 | top1:50.7812 | AUROC:0.5157
101/152 | Loss:0.6927 | top1:53.4576 | AUROC:0.4999
152/152 | Loss:0.6925 | top1:53.3436 | AUROC:0.4994
16/16 | Loss:0.6953 | top1:50.0000 | AUROC:0.5000

Epoch: [4 | 300] LR: 0.310000
1/152 | Loss:0.6914 | top1:54.4922 | AUROC:0.4876
101/152 | Loss:0.6925 | top1:53.1250 | AUROC:0.5019
152/152 | Loss:0.6920 | top1:53.4988 | AUROC:0.5028
16/16 | Loss:0.6973 | top1:50.0000 | AUROC:0.5022

Epoch: [5 | 300] LR: 0.380000
1/152 | Loss:0.6885 | top1:55

16/16 | Loss:0.0166 | top1:99.5897 | AUROC:0.9998

Epoch: [36 | 300] LR: 0.787433
1/152 | Loss:0.0400 | top1:98.8281 | AUROC:0.9989
101/152 | Loss:0.0362 | top1:98.6792 | AUROC:0.9993
152/152 | Loss:0.0358 | top1:98.7337 | AUROC:0.9993
16/16 | Loss:0.0600 | top1:98.2179 | AUROC:0.9995

Epoch: [37 | 300] LR: 0.786370
1/152 | Loss:0.1025 | top1:96.0938 | AUROC:0.9979
101/152 | Loss:0.0465 | top1:98.3756 | AUROC:0.9989
152/152 | Loss:0.0424 | top1:98.5384 | AUROC:0.9990
16/16 | Loss:0.0117 | top1:99.5641 | AUROC:0.9999

Epoch: [38 | 300] LR: 0.785265
1/152 | Loss:0.0281 | top1:99.0234 | AUROC:0.9995
101/152 | Loss:0.0308 | top1:98.9268 | AUROC:0.9994
152/152 | Loss:0.0317 | top1:98.9238 | AUROC:0.9994
16/16 | Loss:0.0207 | top1:99.1667 | AUROC:0.9997

Epoch: [39 | 300] LR: 0.784117
1/152 | Loss:0.0164 | top1:99.4141 | AUROC:0.9999
101/152 | Loss:0.0392 | top1:98.5903 | AUROC:0.9991
152/152 | Loss:0.0399 | top1:98.5617 | AUROC:0.9991
16/16 | Loss:0.0249 | top1:99.0769 | AUROC:0.9998

Epoch

152/152 | Loss:0.0313 | top1:98.8863 | AUROC:0.9994
16/16 | Loss:0.0149 | top1:99.3718 | AUROC:0.9999

Epoch: [71 | 300] LR: 0.726051
1/152 | Loss:0.0355 | top1:98.6328 | AUROC:0.9994
101/152 | Loss:0.0297 | top1:98.9074 | AUROC:0.9995
152/152 | Loss:0.0301 | top1:98.8915 | AUROC:0.9995
16/16 | Loss:0.0307 | top1:99.0128 | AUROC:0.9999

Epoch: [72 | 300] LR: 0.723607
1/152 | Loss:0.0290 | top1:98.8281 | AUROC:0.9996
101/152 | Loss:0.0288 | top1:99.0254 | AUROC:0.9995
152/152 | Loss:0.0308 | top1:98.9368 | AUROC:0.9994
16/16 | Loss:0.0139 | top1:99.6410 | AUROC:0.9998

Epoch: [73 | 300] LR: 0.721127
1/152 | Loss:0.0249 | top1:98.8281 | AUROC:0.9997
101/152 | Loss:0.0298 | top1:98.9577 | AUROC:0.9994
152/152 | Loss:0.0287 | top1:99.0066 | AUROC:0.9995
16/16 | Loss:0.0119 | top1:99.5256 | AUROC:0.9999

Epoch: [74 | 300] LR: 0.718612
1/152 | Loss:0.0173 | top1:99.2188 | AUROC:0.9999
101/152 | Loss:0.0313 | top1:98.9480 | AUROC:0.9994
152/152 | Loss:0.0307 | top1:98.9678 | AUROC:0.9994
16/1

101/152 | Loss:0.0058 | top1:99.7931 | AUROC:1.0000
152/152 | Loss:0.0056 | top1:99.7969 | AUROC:1.0000
16/16 | Loss:0.0035 | top1:99.9231 | AUROC:1.0000

Epoch: [106 | 300] LR: 0.062136
1/152 | Loss:0.0002 | top1:100.0000 | AUROC:1.0000
101/152 | Loss:0.0063 | top1:99.8260 | AUROC:0.9999
152/152 | Loss:0.0055 | top1:99.8383 | AUROC:1.0000
16/16 | Loss:0.0031 | top1:99.8846 | AUROC:1.0000

Epoch: [107 | 300] LR: 0.061786
1/152 | Loss:0.0009 | top1:100.0000 | AUROC:1.0000
101/152 | Loss:0.0057 | top1:99.8240 | AUROC:1.0000
152/152 | Loss:0.0059 | top1:99.8137 | AUROC:1.0000
16/16 | Loss:0.0030 | top1:99.9359 | AUROC:1.0000

Epoch: [108 | 300] LR: 0.061433
1/152 | Loss:0.0147 | top1:99.8047 | AUROC:0.9999
101/152 | Loss:0.0070 | top1:99.7853 | AUROC:1.0000
152/152 | Loss:0.0066 | top1:99.7943 | AUROC:1.0000
16/16 | Loss:0.0032 | top1:99.8974 | AUROC:1.0000

Epoch: [109 | 300] LR: 0.061078
1/152 | Loss:0.0173 | top1:99.6094 | AUROC:0.9998
101/152 | Loss:0.0064 | top1:99.7989 | AUROC:1.000

1/152 | Loss:0.0110 | top1:99.4141 | AUROC:1.0000
101/152 | Loss:0.0069 | top1:99.7679 | AUROC:1.0000
152/152 | Loss:0.0066 | top1:99.7685 | AUROC:1.0000
16/16 | Loss:0.0047 | top1:99.8462 | AUROC:1.0000

Epoch: [141 | 300] LR: 0.048726
1/152 | Loss:0.0047 | top1:99.8047 | AUROC:1.0000
101/152 | Loss:0.0077 | top1:99.7467 | AUROC:0.9999
152/152 | Loss:0.0075 | top1:99.7698 | AUROC:0.9999
16/16 | Loss:0.0026 | top1:99.9231 | AUROC:1.0000

Epoch: [142 | 300] LR: 0.048316
1/152 | Loss:0.0026 | top1:99.8047 | AUROC:1.0000
101/152 | Loss:0.0076 | top1:99.7409 | AUROC:1.0000
152/152 | Loss:0.0068 | top1:99.7698 | AUROC:1.0000
16/16 | Loss:0.0060 | top1:99.8333 | AUROC:1.0000

Epoch: [143 | 300] LR: 0.047906
1/152 | Loss:0.0121 | top1:99.8047 | AUROC:0.9999
101/152 | Loss:0.0073 | top1:99.7679 | AUROC:1.0000
152/152 | Loss:0.0076 | top1:99.7439 | AUROC:1.0000
16/16 | Loss:0.0033 | top1:99.8590 | AUROC:1.0000

Epoch: [144 | 300] LR: 0.047495
1/152 | Loss:0.0019 | top1:100.0000 | AUROC:1.0000
1

16/16 | Loss:0.0039 | top1:99.8846 | AUROC:1.0000

Epoch: [175 | 300] LR: 0.034571
1/152 | Loss:0.0062 | top1:99.8047 | AUROC:1.0000
101/152 | Loss:0.0063 | top1:99.7873 | AUROC:1.0000
152/152 | Loss:0.0063 | top1:99.7853 | AUROC:1.0000
16/16 | Loss:0.0036 | top1:99.8718 | AUROC:1.0000

Epoch: [176 | 300] LR: 0.034157
1/152 | Loss:0.0106 | top1:99.4141 | AUROC:1.0000
101/152 | Loss:0.0060 | top1:99.8008 | AUROC:1.0000
152/152 | Loss:0.0064 | top1:99.7840 | AUROC:1.0000
16/16 | Loss:0.0023 | top1:99.9231 | AUROC:1.0000

Epoch: [177 | 300] LR: 0.003374
1/152 | Loss:0.0124 | top1:99.8047 | AUROC:0.9998
101/152 | Loss:0.0048 | top1:99.8414 | AUROC:1.0000
152/152 | Loss:0.0047 | top1:99.8461 | AUROC:1.0000
16/16 | Loss:0.0029 | top1:99.8718 | AUROC:1.0000

Epoch: [178 | 300] LR: 0.003333
1/152 | Loss:0.0012 | top1:100.0000 | AUROC:1.0000
101/152 | Loss:0.0045 | top1:99.8298 | AUROC:1.0000
152/152 | Loss:0.0044 | top1:99.8357 | AUROC:1.0000
16/16 | Loss:0.0019 | top1:99.9487 | AUROC:1.0000



KeyboardInterrupt: 